**Подключение библиотек и скриптов**

In [37]:
import numpy as np
import pandas as pd

**Пути к директориям и файлам**

In [38]:
DATASET_PATH = 'housing.csv'
PREPARED_DATASET_PATH = 'housing_prepared.csv'

## **Загрузка данных**

**Описание датасета**

Статистические данные о ряде домов в Калифорнии, основанные на переписи 1990 года.

* **longitude** - долгота
* **latitude** - широта
* **housing_median_age** - средний возраст дома
* **total_rooms** - общее количество комнат
* **total_bedrooms** - общее количество спален
* **population** - количество проживающих
* **households** - домохозяйства
* **median_income** - средний доход
* **median_house_value** - средняя стоимость дома
* **ocean_proximity** - близость океана

Считываем датасет и выводим первые пять строк для проверки.

In [39]:
ds_house = pd.read_csv(DATASET_PATH, sep = ',')
ds_house.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,id
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,1
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,2
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,4


Посмотрим общее описание датасета и проанализируем признаки.

In [40]:
ds_house.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,id
count,20640.000000,20640.000000,19918.000000,20640.000000,20433.000000,20041.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.471242,35.036934,28.653630,2635.763081,537.870553,1425.418243,499.539680,3.870671,206855.816909,10319.500000
std,5.041408,94.903955,12.576796,2181.615252,421.385070,1135.185798,382.329753,1.899822,115395.615874,5958.399114
min,-124.350000,-13534.030000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000,0.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,786.000000,280.000000,2.563400,119600.000000,5159.750000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1165.000000,409.000000,3.534800,179700.000000,10319.500000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1726.000000,605.000000,4.743250,264725.000000,15479.250000
max,122.030000,1327.130000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000,20639.000000


Можно сделать несколько предварительных выводов:
1. Не все признаки датасета заполнены: всего 20640 объектов, а, например, количество признаков 'total_bedrooms' и 'population' -
20433 и 20041 соответственно. Значит, какие-то значения пропущены и эти пропуски нужно будет отработать.
2. Минимальные и максимальные значения долготы и широты (по сравнению со средним и квантилями) говорят о том, что имеем артефакты,
которые тоже нужно будет обработать.

**Обработка пропусков**

В каких признаках и сколько пропусков.

In [41]:
len(ds_house) - ds_house.count()

longitude               0
latitude                0
housing_median_age    722
total_rooms             0
total_bedrooms        207
population            599
households              0
median_income           0
median_house_value      0
ocean_proximity         0
id                      0
dtype: int64

Заполняем пропуски медианным значением соответствующего признака (медиана устойчивее прочих способов).

Считаю неверным заполнять "total_bedrooms" медианным значением (т.к. получим результат типа "всего комнат в доме - 154,
из них спален - 435). Нулевых значений в признаке нет, поэтому пока заполню нулями. Дальше обработаю иным способом.

По этой же причине нельзя заполнять медианным значением признак "population", т.к. можем получить "1165 человек в 5 комнатах".
Аналогичный расчёт ниже.

In [42]:
median = ds_house['housing_median_age'].median()
ds_house['housing_median_age'].fillna(median, inplace = True)

ds_house['total_bedrooms'].fillna(0, inplace = True)

ds_house['population'].fillna(0, inplace = True)

**Обработка номинативного признака**

In [43]:
ds_house['ocean_proximity'].value_counts()

<1H OCEAN     9127
INLAND        6542
NEAR OCEAN    2655
NEAR BAY      2288
-               23
ISLAND           5
Name: ocean_proximity, dtype: int64

23 пропущенных значения заменяем модой.

In [44]:
ds_house.replace({'ocean_proximity': {'-': ds_house['ocean_proximity'].mode()[0]}}, inplace = True)

## **Обработка выбросов**

California City находится в северном и западном полушарии и имеет координаты:
широта: 35.1258000
долгота: -117.9859000

In [45]:
ds_house[ds_house['longitude'] >= 0]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,id
3479,118.51,34.29,29.0,1287.0,194.0,525.0,187.0,6.4171,319300.0,<1H OCEAN,3479
5904,118.43,34.29,39.0,1769.0,410.0,1499.0,390.0,3.1212,153500.0,<1H OCEAN,5904
8405,118.36,33.93,40.0,1625.0,500.0,2036.0,476.0,2.6298,156500.0,<1H OCEAN,8405
8636,118.41,33.88,43.0,2492.0,449.0,1033.0,437.0,7.9614,500001.0,<1H OCEAN,8636
13051,121.29,38.61,17.0,13553.0,2474.0,6544.0,2359.0,3.9727,132700.0,INLAND,13051
15263,117.27,33.02,21.0,2144.0,340.0,928.0,344.0,5.7980,286100.0,NEAR OCEAN,15263
17085,0.00,37.47,33.0,1266.0,415.0,1991.0,334.0,2.9200,202800.0,NEAR OCEAN,17085
17359,0.00,34.88,4.0,3680.0,559.0,1678.0,569.0,5.0639,201700.0,<1H OCEAN,17359
18551,122.03,36.96,28.0,1607.0,421.0,926.0,385.0,2.4250,216100.0,NEAR OCEAN,18551
19423,0.00,37.69,5.0,9601.0,1639.0,4449.0,1575.0,4.5332,195500.0,INLAND,19423


Видно, что либо потерян минус, либо долгота неизвестна. Заменяем ноль медианным значением.

In [46]:
ds_house.loc[ds_house['longitude'] > 0, 'longitude'] = ds_house.loc[ds_house['longitude'] > 0, 'longitude'] * -1
ds_house.loc[ds_house['longitude'] == 0, 'longitude'] = ds_house['longitude'].median()

Аналогично обработаем широту: корректные значение широты должны быть 35 плюс-минус несколько градусов.

In [47]:
ds_house[(ds_house['latitude'] <= 0) | (ds_house['latitude'] > 50)]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,id
8283,-118.13,-13534.03,45.0,1016.0,172.0,361.0,163.0,7.5000,434500.0,NEAR OCEAN,8283
12772,-121.42,1327.13,29.0,2217.0,536.0,1203.0,507.0,1.9412,73100.0,INLAND,12772


In [48]:
ds_house.loc[(ds_house['latitude'] <= 0) | (ds_house['latitude'] > 50), 'latitude'] = ds_house['latitude'].median()

Исключаем ненужный признак id

In [49]:
ds_house.drop(columns = 'id', inplace = True)

Преобразуем категориальный признак "oceans_proximity" в несколько бинарных

In [50]:
ds_house = pd.concat([ds_house, pd.get_dummies(ds_house['ocean_proximity'])], axis = 1)
ds_house.drop(columns = 'ocean_proximity', inplace = True)

Рассчитаем дополнительные параметры:

In [51]:
# Доля спален в общем количестве комнат
ds_house['bedroom_share'] = ds_house['total_bedrooms'] / ds_house['total_rooms'] * 100

# Сколько человек в срелнем в одном домохозяйстве
ds_house['men_per_household'] = ds_house['population'] / ds_house['households']


И вот только сейчас заполняем пропуски признака "total_bedrooms", рассчитывая их количество через медианное значение доли спален.

In [52]:
median = ds_house[ds_house['bedroom_share'] > 0].median()['bedroom_share']
ds_house.loc[(ds_house['bedroom_share'] == 0), 'total_bedrooms'] = round(median * ds_house['total_rooms'])
ds_house.loc[(ds_house['bedroom_share'] == 0), 'bedroom_share'] = ds_house['total_bedrooms'] / ds_house['total_rooms'] * 100

Аналогично рассчитываем пропуски признака "population" через медианное значение человек в домохозяйстве.

In [53]:
median = ds_house[ds_house['men_per_household'] > 0].median()['men_per_household']
ds_house.loc[(ds_house['men_per_household'] == 0), 'population'] = round(median * ds_house['households'])
ds_house.loc[(ds_house['men_per_household'] == 0), 'men_per_household'] = ds_house['population'] / ds_house['households']


In [54]:
# Сколько человек в среднем живут в одной комнате
ds_house['population_per_room'] = ds_house['population'] / ds_house['total_rooms']

In [63]:
ds_house.nlargest(10, 'population_per_room')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN,bedroom_share,men_per_household,population_per_room,age_cat
16420,-121.29,37.89,26.0,161.0,27.0,1542.0,30.0,5.7485,162500.0,0,1,0,0,0,16.770186,51.400000,9.577640,4
19435,-121.04,37.67,16.0,19.0,19.0,166.0,9.0,0.5360,162500.0,0,1,0,0,0,100.000000,18.444444,8.736842,3
12104,-117.33,33.97,8.0,152.0,19.0,1275.0,20.0,1.6250,162500.0,0,1,0,0,0,12.500000,63.750000,8.388158,2
20352,-119.09,34.22,8.0,40.0,10.0,309.0,16.0,4.0208,52500.0,0,0,0,0,1,25.000000,19.312500,7.725000,2
15790,-122.40,37.77,52.0,144.0,63.0,1061.0,68.0,4.3958,225000.0,0,0,0,1,0,43.750000,15.602941,7.368056,4
13366,-117.63,33.94,36.0,447.0,95.0,2886.0,85.0,4.2578,183300.0,0,1,0,0,0,21.252796,33.952941,6.456376,4
16643,-120.65,35.32,20.0,626.0,212.0,3574.0,261.0,1.0298,300000.0,0,0,0,0,1,33.865815,13.693487,5.709265,3
8874,-118.45,34.06,52.0,204.0,34.0,1154.0,28.0,9.3370,500001.0,1,0,0,0,0,16.666667,41.214286,5.656863,4
19524,-121.00,37.64,19.0,121.0,41.0,658.0,41.0,0.9573,162500.0,0,1,0,0,0,33.884298,16.048780,5.438017,3
4861,-118.28,34.02,29.0,515.0,229.0,2690.0,217.0,0.4999,500001.0,1,0,0,0,0,44.466019,12.396313,5.223301,4


Видно, что 5 объектов с максимальным количеством человек на комнату сильно выбиваются из оставшегося ряда.
Делаю вывод, что это какие-то ошибочные значения (либо какие-то производственные помещения), которые для построения модели не нужны. Удаляю их.

In [64]:
ds_house = ds_house.loc[ds_house['population_per_room'] < 10]

Разделим дома по возрасту на категории.

In [65]:
def age_to_cat(X):

    X['age_cat'] = 0

    X.loc[X['housing_median_age'] <= 5, 'age_cat'] = 1
    X.loc[(X['housing_median_age'] > 5) & (X['housing_median_age'] <= 10), 'age_cat'] = 2
    X.loc[(X['housing_median_age'] > 10) & (X['housing_median_age'] <= 25), 'age_cat'] = 3
    X.loc[X['housing_median_age'] > 25, 'age_cat'] = 4

    return X

In [66]:
ds_house = age_to_cat(ds_house)

In [67]:
ds_house.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN,bedroom_share,men_per_household,population_per_room,age_cat
count,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000,20635.000000
mean,-119.569061,35.631234,28.663775,2636.355658,1054.861885,1423.653986,499.652968,3.870136,206860.647492,0.443421,0.316840,0.000242,0.110880,0.128616,41.470776,2.938384,0.589361,3.498280
std,2.003559,2.135649,12.353131,2181.545323,6301.122420,1130.960762,382.306343,1.899433,115399.563575,0.496801,0.465256,0.015565,0.313991,0.334783,200.424073,1.078783,0.297179,0.706342
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.750000,0.018109,1.000000
25%,-121.800000,33.930000,19.000000,1448.000000,297.000000,786.000000,280.000000,2.562500,119600.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.570616,2.440954,0.436170,3.000000
50%,-118.490000,34.260000,29.000000,2127.000000,438.000000,1165.000000,409.000000,3.534500,179700.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.371669,2.817620,0.516425,4.000000
75%,-118.010000,37.710000,37.000000,3148.000000,657.000000,1724.000000,605.000000,4.742600,264750.000000,1.000000,1.000000,0.000000,0.000000,0.000000,24.124533,3.263586,0.654531,4.000000
max,-114.310000,41.950000,52.000000,39320.000000,237883.000000,35682.000000,6082.000000,15.000100,500001.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2031.818182,63.750000,9.577640,4.000000


Проверим строки с большим общим количеством комнат - выглядят подозрительно.

In [68]:
ds_house.nlargest(10, "total_rooms")

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN,bedroom_share,men_per_household,population_per_room,age_cat
13139,-121.44,38.43,29.0,39320.0,6210.0,16305.0,5358.0,4.9516,153700.0,0,1,0,0,0,15.793489,3.043113,0.414674,4
10309,-117.74,33.89,4.0,37937.0,5471.0,16122.0,5189.0,7.4947,366300.0,1,0,0,0,0,14.421277,3.106957,0.424968,1
9880,-121.79,36.64,11.0,32627.0,6445.0,28566.0,6082.0,2.3087,118800.0,1,0,0,0,0,19.753578,4.696810,0.875533,3
6057,-117.78,34.03,8.0,32054.0,5290.0,15507.0,5050.0,6.0191,253900.0,1,0,0,0,0,16.503401,3.070693,0.483777,2
12201,-117.20,33.58,2.0,30450.0,5033.0,9419.0,3197.0,4.5936,174300.0,1,0,0,0,0,16.528736,2.946200,0.309327,1
9019,-118.78,34.16,9.0,30405.0,4093.0,12873.0,3931.0,8.0137,399200.0,0,0,0,0,1,13.461602,3.274739,0.423384,2
12215,-117.12,33.52,4.0,30401.0,4957.0,13251.0,4339.0,4.5841,212300.0,1,0,0,0,0,16.305385,3.053929,0.435874,1
922,-121.92,37.53,7.0,28258.0,3864.0,12203.0,3701.0,8.4045,451100.0,1,0,0,0,0,13.674004,3.297217,0.431842,2
12623,-121.53,38.48,5.0,27870.0,5027.0,11935.0,4855.0,4.8811,212200.0,0,1,0,0,0,18.037316,2.458290,0.428238,1
6066,-117.87,34.04,7.0,27700.0,4179.0,15037.0,4072.0,6.6288,339700.0,1,0,0,0,0,15.086643,3.692780,0.542852,2


Количество человек на комнату не превышает средних значений, в среднем 3 человека в домохозяйстве (семье), зато количество спален очень мало.
С учётом большого количества комнат делаю вывод, что это здания типа наших общежитий, где проживают семьи с одним ребенком, а не ошибочные данные.

**Сохранение результатов**

In [69]:
ds_house.to_csv(PREPARED_DATASET_PATH, index = False, encoding = 'utf-8')